In [8]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 2], 'x')
y = tf.layers.batch_normalization(x, training=True)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
y_out = sess.run(y, feed_dict={x: [[1.1, 0.9], [0, 0], [-1.1, -0.9]]})

print(y_out)
sess.close()




[[ 1.2239865  1.2236124]
 [ 0.         0.       ]
 [-1.2239865 -1.2236124]]
